In [1]:
import cv2
import numpy as np
import tensorflow as tf

In [2]:
model = tf.keras.models.load_model("Transfer-Resnet50_4.model" , compile = False)

In [3]:
def prepare(ima):
    IMG_SIZE = 100  # 50 in txt-based
    img_array = ima*255
    img_array=img_array/255.0  # filepathread in the image, convert to grayscale
    new_array = cv2.resize(img_array, (IMG_SIZE*3, IMG_SIZE))  # resize image to match model's expected sizing
    return new_array.reshape(-1,IMG_SIZE, IMG_SIZE,3)

In [4]:
def func(image,image1):
    new1 = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    dist = cv2.distanceTransform(new1, cv2.DIST_L2, 5)
    cv2.normalize(dist, dist, 0, 1.0, cv2.NORM_MINMAX)
    return image1,dist

In [5]:
img=cv2.imread("DTDataset/1/1.jpg")
print(img.shape)

(100, 100, 3)


In [6]:
def callback(x):
    pass
cv2.namedWindow('image1')
cx=100
cy=100
rw=300
rh=300
cap=cv2.VideoCapture(0)
ilowH = 0
ihighH = 255
ilowS = 0
ihighS = 255
ilowV = 0
ihighV = 255
cv2.createTrackbar('lowH1', 'image1', ilowH, 255, callback)
cv2.createTrackbar('highH1', 'image1', ihighH, 255, callback)
cv2.createTrackbar('lowS1', 'image1', ilowS, 255, callback)
cv2.createTrackbar('highS1', 'image1', ihighS, 255, callback)
cv2.createTrackbar('lowV1', 'image1', ilowV, 255, callback)
cv2.createTrackbar('highV1', 'image1', ihighV, 255, callback)
while True:
    ret,frame=cap.read()
    ilowH1 = cv2.getTrackbarPos('lowH1', 'image1')
    ihighH1 = cv2.getTrackbarPos('highH1', 'image1')
    ilowS1 = cv2.getTrackbarPos('lowS1', 'image1')
    ihighS1 = cv2.getTrackbarPos('highS1', 'image1')
    ilowV1 = cv2.getTrackbarPos('lowV1', 'image1')
    ihighV1 = cv2.getTrackbarPos('highV1', 'image1')
    hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    low1=np.array([ilowH1,ilowS1,ilowV1])
    high1=np.array([ihighH1,ihighS1,ihighV1])
    img_mask1=cv2.inRange(hsv,low1,high1)
    output1=cv2.bitwise_and(frame,frame,mask=img_mask1)
    cv2.imshow("image1",frame)
    pre=output1[cx:rw,cy:rh]
    image,dist=func(pre,frame)
    #print(dist.shape)
    cv2.imshow('dist',dist)
    prediction = model.predict([prepare(dist)])
    prediction=np.argmax(prediction)
    x1=str(prediction)
    cv2.putText(frame,x1,(60,80),cv2.FONT_HERSHEY_SIMPLEX,3.0,(255,255,255),lineType=cv2.LINE_AA)
    cv2.rectangle(frame,(cx,cy),(cx+rw,cy+rh),(255,255,255),5)
    cv2.putText(output1,x1,(60,80),cv2.FONT_HERSHEY_SIMPLEX,3.0,(255,255,255),lineType=cv2.LINE_AA)
    cv2.rectangle(output1,(cx,cy),(cx+rw,cy+rh),(255,255,255),5)
    cv2.imshow("image",output1)
    cv2.imshow("image1",frame)
    if (cv2.waitKey(1) == 13):
        break
cv2.destroyAllWindows()
cap.release()